In [2]:
import pandas as pd
import re

In [3]:
data = pd.read_json('../data/apartment.json')
data

,address,price,properties
0,"West Wharf | 60 Wharf Dr, Brooklyn, NY","$3,575+ Studio","$4,308+ 1 bd$8,250+ 2 bds$9,167+ 3 bds"
1,"Avalon Willoughby Square | 214 Duffield St, Br...","$3,254+ Studio","$3,864+ 1 bd"
2,"AVA DoBro | 100 Willoughby St, Brooklyn, NY","$3,600+ Studio","$4,451+ 1 bd$5,462+ 2 bds$6,075+ 3 bds"
3,"THE BELLSLIP | 1 Bell Slip, Brooklyn, NY","$3,204+ Studio","$4,930+ 1 bd$7,094+ 2 bds"
4,"AVA Fort Greene | 343 Gold St, Brooklyn, NY","$3,775+ Studio","$3,835+ 1 bd$5,105+ 2 bds$7,832+ 3 bds"
...,...,...,...
175,"203 17th St APT 19, Brooklyn, NY 11215","$2,450/mo",1 bd1 ba-- sqft - Apartment for rent
176,"308 Court St #2, Brooklyn, NY 11231","$3,500/mo",1 bd1 ba-- sqft - Apartment for rent
177,"1520 Fulton St APT 7E, Brooklyn, NY 11216","$3,995/mo",2 bds1 ba785 sqft - Apartment for rent
178,"691 Chauncey St, Brooklyn, NY","$4,000+ 3 bds","$5,650+ 5 bds"


In [6]:
new_data = pd.DataFrame(columns=["ApartmentID", "Price", "NumberOfBedrooms", "NumberOfBathrooms", "Address"])
new_data

,ApartmentID,Price,NumberOfBedrooms,NumberOfBathrooms,Address


In [7]:
new_index = 0

for index, row in data.iterrows():
    if ("bd" in row["price"] or "bds" in row["price"]):
        # Extract the prices
        prices = re.findall(r'\$([\d,]+)\+', row["price"])
        
        # Extract the number of bedrooms
        bedrooms = re.findall(r'(\d+)\s*bd', row["price"])
        
        if (row["properties"] is not None and isinstance(row["properties"], str)):
            new_prices = re.findall(r'\$([\d,]+)\+', row["properties"])
            
            prices.extend(new_prices)
            
            new_bedrooms = re.findall(r'(\d+)\s*bd', row["properties"])
            
            bedrooms.extend(new_bedrooms)
        
        for i in range(len(prices)):
            
            # Removing , from price
            prices[i] = prices[i].replace(",", "")
            
            # Because it's an apartment so it is guarantee to have at least one bathroom
            bathroom = 1
            
            # Inserting new row to the processed data
            new_data.loc[new_index] = [new_index+1, int(prices[i]), int(bedrooms[i]), bathroom, row["address"]]
            new_index += 1
            
    elif ("Studio" in row["price"]):
        # Extract the prices
        prices = re.findall(r'\$([\d,]+)\+', row["price"])
        
        bedrooms = [1]
        
        if (row["properties"] is not None and isinstance(row["properties"], str)):
            new_prices = re.findall(r'\$([\d,]+)\+', row["properties"])
            
            prices.extend(new_prices)
            
            new_bedrooms = re.findall(r'(\d+)\s*bd', row["properties"])
            
            bedrooms.extend(new_bedrooms)
        
        for i in range(len(prices)):
            
            # Removing , from price
            prices[i] = prices[i].replace(",", "")
            
            # Studio type apartment only have one bathroom and bedroom
            bathroom = 1
            bedroom = 1
            
            # Inserting new row to the processed data
            new_data.loc[new_index] = [new_index+1, int(prices[i]), int(bedrooms[i]), bathroom, row["address"]]
            new_index += 1
    
    else:
        # Replace all unnecessary character in price
        price = row["price"].replace("/mo", "")
        price = price.replace(",", "")
        price = price.replace("$", "")
        
        # Split the string based on variations of 'bds', 'bd', 'ba', and 'bas'
        pattern = r'(\d+)\s+(bds|bd|ba|bas)\b'
        hasil = re.findall(pattern, row["properties"])
        
        result = {}
        for item in hasil:
            if item[1] in result:
                result[item[1]] = int(result[item[1]]) + int(item[0])
            else:
                result[item[1]] = int(item[0])
        
        bedroom = 0
        bathroom = 0
        
        # Get the value of how many bedroom and bathroom at the apartment
        if "bd" in result:
            bedroom = result["bd"]
        elif "bds" in result:
            bedroom = result["bds"]
        
        if "ba" in result:
            bathroom = result["ba"]
        elif "bas" in result:
            bathroom = result["bas"]
        
        # Inserting new row to the processed data
        new_data.loc[new_index] = [new_index+1, int(price), bedroom, bathroom, row["address"]]
        new_index += 1
        
new_data

,ApartmentID,Price,NumberOfBedrooms,NumberOfBathrooms,Address
0,1,3575,1,1,"West Wharf | 60 Wharf Dr, Brooklyn, NY"
1,2,4308,1,1,"West Wharf | 60 Wharf Dr, Brooklyn, NY"
2,3,8250,2,1,"West Wharf | 60 Wharf Dr, Brooklyn, NY"
3,4,9167,3,1,"West Wharf | 60 Wharf Dr, Brooklyn, NY"
4,5,3254,1,1,"Avalon Willoughby Square | 214 Duffield St, Br..."
...,...,...,...,...,...
254,255,3500,0,1,"308 Court St #2, Brooklyn, NY 11231"
255,256,3995,0,0,"1520 Fulton St APT 7E, Brooklyn, NY 11216"
256,257,4000,3,1,"691 Chauncey St, Brooklyn, NY"
257,258,5650,5,1,"691 Chauncey St, Brooklyn, NY"


In [9]:
# Save PreProcessed Data to Excel
new_data.to_json('../data/proccessed_data.json', orient='records', indent=4)